In [1]:
import json

import pandas as pd
import numpy as np

import re

In [6]:
raw_data_json = json.load(open("../../AllCards.json"))

In [7]:
i = 0
for card in raw_data_json:
    print(raw_data_json[card])
    i += 1
    if i > 10:
        break

{'colorIdentity': ['G', 'R'], 'colors': ['G', 'R'], 'convertedManaCost': 6.0, 'foreignData': [], 'isReserved': False, 'layout': 'normal', 'legalities': {}, 'manaCost': '{2}{R}{R}{G}{G}', 'name': '"Ach! Hans, Run!"', 'printings': ['UNH'], 'rulings': [], 'subtypes': [], 'supertypes': [], 'text': 'At the beginning of your upkeep, you may say "Ach Hans, run It\'s the . . ." and the name of a creature card. If you do, search your library for a card with that name, put it onto the battlefield, then shuffle your library. That creature gains haste. Exile it at the beginning of the next end step.', 'type': 'Enchantment', 'types': ['Enchantment'], 'uuid': '84f2c8f5-8e11-4639-b7de-00e4a2cbabee'}
{'colorIdentity': ['B'], 'colors': ['B'], 'convertedManaCost': 3.0, 'foreignData': [], 'isReserved': False, 'layout': 'normal', 'legalities': {}, 'manaCost': '{2}{B}', 'name': '"Rumors of My Death . . ."', 'printings': ['UST'], 'rulings': [], 'subtypes': [], 'supertypes': [], 'text': '{3}{B}, Exile a perm

In [8]:
raw_data_json["AWOL"]

{'colorIdentity': ['W'],
 'colors': ['W'],
 'convertedManaCost': 3.0,
 'foreignData': [],
 'isReserved': False,
 'layout': 'normal',
 'legalities': {},
 'manaCost': '{2}{W}',
 'name': 'AWOL',
 'printings': ['UNH'],
 'rulings': [],
 'subtypes': [],
 'supertypes': [],
 'text': 'Exile target attacking creature. Then remove it from the game. Then put it into the absolutely-removed-from-the-freaking-game-forever zone.',
 'type': 'Instant',
 'types': ['Instant'],
 'uuid': 'fcf3447a-ec7d-47d7-bf6b-e2bbe0ac58df'}

In [9]:
raw_data_json["AWOL"].get("type", 0)

'Instant'

In [10]:
len(raw_data_json.keys())

19267

In [27]:
creature_names_ls = [card for card in raw_data_json if "Creature" in raw_data_json[card].get("type", 0)]

In [28]:
len(creature_names_ls)

10062

In [44]:
# Format for feature vectors will be power, toughness, flying, firststrike, doublestrike, trample, 
# vigilance, defender, haste, menace, fear, deathtouch, indestructible, hexproof/shroud
columns = ["name", "cid_c", "cid_w", "cid_u", "cid_g", "cid_b", "cid_r", "mc_c", "mc_w", "mc_u", "mc_b", "mc_r", "mc_g",\
           "power", "toughness", "flying", "trample", "doublestrike", "first_strike", "vigilance", "haste", \
           "menace", "fear", "shadow", "defender", "deathtouch", "indestructible", "hexproof", "shroud"]

In [16]:
# test regex behavior re (har!) casting cost
test_cost = raw_data_json["Wrath of God"].get("manaCost", "")
test_cost_2 = '{2}{W}{W}{W}'

In [17]:
matches = re.findall("{W}", test_cost)

In [18]:
len(matches)

2

In [19]:
matches = re.search("{(\d*)}", test_cost_2)

In [20]:
int(matches.groups()[0])

2

In [45]:
def produce_vector(card):
    name = card.get("name", "")
    
    cid_c = int(len(card.get("colorIdentity", [])) == 0)
    cid_w = int('W' in card.get("colorIdentity", []))
    cid_u = int('U' in card.get("colorIdentity", []))
    cid_b = int('B' in card.get("colorIdentity", []))
    cid_r = int('R' in card.get("colorIdentity", []))
    cid_g = int('G' in card.get("colorIdentity", []))
    flying = int("flying" in card.get("text", "").lower())
    trample = int("trample" in card.get("text", "").lower())
    double_strike = int("double strike" in card.get("text", "").lower())
    first_strike = int("first strike" in card.get("text", "").lower())
    vigilance = int("vigilance" in card.get("text", "").lower())
    haste = int("haste" in card.get("text", "").lower())
    menace = int("menace" in card.get("text", "").lower())
    fear = int("fear" in card.get("text", "").lower())
    shadow = int("shadow" in card.get("text", "").lower())
    defender = int("defender" in card.get("text", "").lower())
    deathtouch = int("deathtouch" in card.get("text", "").lower())
    indestructible = int("indestructible" in card.get("text", "").lower())
    hexproof = int("hexproof" in card.get("text", "").lower())
    shroud = int("shroud" in card.get("text", "").lower())

    power = int(card.get("power", "").lower())
    toughness = int(card.get("toughness", "").lower())

    mc_c = 0
    mc_c_result = re.search("{(\d*)}", card.get("manaCost", ""))
    if mc_c_result is not None:
        mc_c = int(mc_c_result.groups()[0])
    mc_w = len(re.findall("{W}", card.get("manaCost", "")))
    mc_u = len(re.findall("{U}", card.get("manaCost", "")))
    mc_b = len(re.findall("{B}", card.get("manaCost", "")))
    mc_r = len(re.findall("{R}", card.get("manaCost", "")))
    mc_g = len(re.findall("{G}", card.get("manaCost", "")))
    
    return [name, cid_c , cid_w , cid_u , cid_b ,cid_r , cid_g , mc_c, mc_w, mc_u, mc_b, mc_r, mc_g, \
            power, toughness, flying , trample , double_strike ,\
            first_strike , vigilance , haste , menace , fear , shadow ,\
            defender , deathtouch , indestructible , hexproof , shroud ]

In [46]:
list(zip(columns, produce_vector(raw_data_json["Wall of Blossoms"])))

[('name', 'Wall of Blossoms'),
 ('cid_c', 0),
 ('cid_w', 0),
 ('cid_u', 0),
 ('cid_g', 0),
 ('cid_b', 0),
 ('cid_r', 1),
 ('mc_c', 1),
 ('mc_w', 0),
 ('mc_u', 0),
 ('mc_b', 0),
 ('mc_r', 0),
 ('mc_g', 1),
 ('power', 0),
 ('toughness', 4),
 ('flying', 0),
 ('trample', 0),
 ('doublestrike', 0),
 ('first_strike', 0),
 ('vigilance', 0),
 ('haste', 0),
 ('menace', 0),
 ('fear', 0),
 ('shadow', 0),
 ('defender', 1),
 ('deathtouch', 0),
 ('indestructible', 0),
 ('hexproof', 0),
 ('shroud', 0)]

In [47]:
creature_vectors_ls = []
for creature_name in creature_names_ls:
    try:
        creature_vectors_ls.append(produce_vector(raw_data_json[creature_name]))
    except:
        print("Error with " + creature_name)
    

Error with Adamaro, First to Desire
Error with Aeon Chronicler
Error with Allosaurus Rider
Error with Altar Golem
Error with An-Havva Constable
Error with Ancient Ooze
Error with Angry Mob
Error with Apocalypse Demon
Error with Aquamorph Entity
Error with Assquatch
Error with Avatar of Me
Error with Aven Trailblazer
Error with Awakened Amalgam
Error with Aysen Crusader
Error with B.F.M. (Big Furry Monster)
Error with Bad Ass
Error with Bad Ass (b)
Error with Battle Squadron
Error with Beast of Burden
Error with Benalish Commander
Error with Boneyard Wurm
Error with Broodstar
Error with Caller of the Hunt
Error with Cantivore
Error with Cardpecker
Error with Cardpecker (b)
Error with Chameleon Spirit
Error with Cheap Ass
Error with Cognivore
Error with Coiling Woodworm
Error with Consuming Aberration
Error with Crackling Drake
Error with Crowd of Cinders
Error with Crusader of Odric
Error with Dakkon Blackblade
Error with Dakmor Sorceress
Error with Darksteel Juggernaut
Error with Daunt

In [48]:
creature_vectors_df = pd.DataFrame(creature_vectors_ls)

In [49]:
creature_vectors_df.columns = columns

In [50]:
creature_vectors_df

,name,cid_c,cid_w,cid_u,cid_g,cid_b,cid_r,mc_c,mc_w,mc_u,...,vigilance,haste,menace,fear,shadow,defender,deathtouch,indestructible,hexproof,shroud
0,Abattoir Ghoul,0,0,0,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abbey Gargoyles,0,1,0,0,0,0,2,3,0,...,0,0,0,0,0,0,0,0,0,0
2,Abbey Griffin,0,1,0,0,0,0,3,1,0,...,1,0,0,0,0,0,0,0,0,0
3,Abbey Matron,0,1,0,0,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Abbot of Keral Keep,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Aberrant Researcher,0,0,1,0,0,0,3,0,1,...,0,0,0,0,0,0,0,0,0,0
6,Abhorrent Overlord,0,0,0,1,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Abolisher of Bloodlines,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Abomination,0,0,0,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Abomination of Gudul,0,0,1,1,0,1,3,0,1,...,0,0,0,0,0,0,0,0,0,0


In [51]:
creature_vectors_df.to_csv("../data/processed.csv")